In [20]:
import pandas as pd
Inflows = pd.read_parquet("/uss/hdsi-prismdata/q1-ucsd-inflows.pqt")
Outflows = pd.read_parquet("/uss/hdsi-prismdata/q1-ucsd-outflows.pqt")

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
Outflows

,prism_consumer_id,prism_account_id,memo,amount,posted_date,category
0,0,acc_0,LOAN,900.60,2022-07-05,LOAN
1,0,acc_0,ATM_CASH,80.00,2022-03-25,ATM_CASH
2,0,acc_0,TST* Casa Del Rio - Exp Fairlawn OH 09/24,18.42,2022-09-26,FOOD_AND_BEVERAGES
3,0,acc_0,LOAN,634.00,2023-01-10,LOAN
4,0,acc_0,Buffalo Wild Wings,26.47,2022-09-12,FOOD_AND_BEVERAGES
...,...,...,...,...,...,...
2597483,5941,acc_9524,ATM_CASH,8.42,2023-01-25,ATM_CASH
2597484,5941,acc_9524,ATM_CASH,2.06,2023-01-25,ATM_CASH
2597485,5941,acc_9524,ATM_CASH,262.88,2023-01-25,ATM_CASH
2597486,5941,acc_9524,ATM_CASH,10.00,2023-01-25,ATM_CASH


In [23]:
cat = list(Outflows["category"].value_counts().index)
to_see = Outflows[["category", "memo"]].value_counts()
output = {}
for i in cat:
    output[i] = list(to_see[i].index)

In [24]:
cat[0]

'GENERAL_MERCHANDISE'

In [33]:
Outflows['memo'].shape[0]

2597488

In [34]:
Outflows[Outflows['memo'] == Outflows['category']]['memo'].shape[0]

1291036

In [109]:
Diff = Outflows[Outflows['memo'] != Outflows['category']]['memo']
m = Diff.str.contains(r'(?:#)?X{3,}', na=False)
matched = Diff[m]
unmatched = Diff[~m]

In [110]:
matched

11                            TGI FRIDAYS XXXX STOW OH 12/31
17                            HARDEES XXXXXXX AKRON OH 05/29
18               MARKET DI XXXX State Cuyahoga Fall OH 04/06
33               LONGHORN STEAK XXXXX CUYAHOGA FALL OH 10/09
55                 GIANT-EAG XXXX Corpora Uniontown OH 09/28
                                 ...                        
2597457    DEBIT CARD WITHDRAWAL PURCHASEAmazon Prime*TI4...
2597462    POS WITHDRAWALAZ LOT QUIKTRIP XXXX XXXX E INDI...
2597465    POS WITHDRAWALWAL-MART #XXXX XXXX E MCKELLIPS ...
2597468    WITHDRAWAL Salt River ProjeTYPE: ONLINE PMT CO...
2597476    POS WITHDRAWALFRYS-FOOD-DRG #1 435 S. E MESA A...
Name: memo, Length: 653578, dtype: object

In [111]:
unmatched

2              TST* Casa Del Rio - Exp Fairlawn OH 09/24
4                                     Buffalo Wild Wings
6                                        Oculus CA 04/16
7                            LOS GIRASOLES STOW OH 03/08
8                              BUZZIS LAUNDRY 1 OH 03/28
                               ...                      
2596634            DEBIT CARD KING TACO 30 BELLFLOWER CA
2596652        DEBIT CARD DLR NEW ORLEANS HOL ANAHEIM CA
2596653     DEBIT CARD ROUND TABLE PIZZA APPLE VALLEY CA
2596655         DEBIT CARD DLR RONTO ROASTERS ANAHEIM CA
2596683    RECURRING DEBIT CARD CLEO AI HTTPSWWW.MEET DE
Name: memo, Length: 652874, dtype: object

In [114]:
import re

pat = re.compile(
    r"X{3,}"
)

In [115]:
Diff = Diff.reset_index()

In [116]:
Diff

,index,memo
0,2,TST* Casa Del Rio - Exp Fairlawn OH 09/24
1,4,Buffalo Wild Wings
2,6,Oculus CA 04/16
3,7,LOS GIRASOLES STOW OH 03/08
4,8,BUZZIS LAUNDRY 1 OH 03/28
...,...,...
1306447,2597457,DEBIT CARD WITHDRAWAL PURCHASEAmazon Prime*TI4...
1306448,2597462,POS WITHDRAWALAZ LOT QUIKTRIP XXXX XXXX E INDI...
1306449,2597465,POS WITHDRAWALWAL-MART #XXXX XXXX E MCKELLIPS ...
1306450,2597468,WITHDRAWAL Salt River ProjeTYPE: ONLINE PMT CO...


In [120]:
import re
import pandas as pd
Inflows = pd.read_parquet("/uss/hdsi-prismdata/q1-ucsd-inflows.pqt")
Outflows = pd.read_parquet("/uss/hdsi-prismdata/q1-ucsd-outflows.pqt")
Diff = Outflows[Outflows['memo'] != Outflows['category']]['memo']
# Compile once for speed
RULES = [
    # 0) Normalize whitespace & weird spaces first
    (re.compile(r"\u00A0"), " "),                                # non-breaking space
    (re.compile(r"\s{2,}"), " "),
    
    # 1) Remove mask tokens & generic IDs
    (re.compile(r"(?:#)?X{4,}", re.I), " "),                     # XXXX, #XXXX, XXXXXXXXX...
    (re.compile(r"\bRF#\w+\b", re.I), " "),
    (re.compile(r"\bMCC\b\s*\w+", re.I), " "),
    (re.compile(r"\bSEQ#?\s*\w+\b", re.I), " "),
    (re.compile(r"\bC#\**\s*\w+\b", re.I), " "),
    (re.compile(r"\bID:\s*\w+\b", re.I), " "),
    (re.compile(r"\bOrder Number\b.*?(?=\b[A-Z]|$)", re.I), " "),
    
    # 2) Remove phone numbers (real or masked)
    (re.compile(r"\b(?:\d{3}-\d{3}-\d{4}|XXX-XXX-XXXX)\b", re.I), " "),
    (re.compile(r"\b\d{3}-\d{4}\b"), " "),                       # 888-802-30 (truncated)
    
    # 3) Remove dates & times
    (re.compile(r"\b\d{2}[/-]\d{2}(?:[/-]\d{2,4})?\b"), " "),    # 03/25, 03/25/22, 03-25-22
    (re.compile(r"\b\d{1,2}:\d{2}(?::\d{2})?\s*[ap]?\b", re.I), " "),  # 12:03, 07:07p
    (re.compile(r"\bDATE\b\s*\d{2}[/-]\d{2}\b", re.I), " "),
    (re.compile(r"\(\d{2}/\d{2}/\d{2}.*?\)"), " "),              # parenthetical stamps
    
    # 4) Remove URLs/domains & billing tails
    (re.compile(r"\b(?:https?:\/\/)?[A-Za-z0-9.-]+\.(?:com|org|net|edu|io|co)\b(?:\/[^\s]*)?", re.I), " "),
    (re.compile(r"\b(?:AMZN\.COM\/BILL|AMZNAMZN\.COM\s*BILL|AMAZON\.COM\/BILL|APPLE\.COM\/BILL|G\.CO\/HELPPAY#|SUPPORT\.GOOGLE?)\b", re.I), " "),
    
    # 5) Remove banking boilerplate & transport words
    (re.compile(r"\b(?:PURCHASE|AUTHORIZED|RECURRING|PAYMENT|WITHDRAWAL|DEBIT|CREDIT|POS|CHECKCARD|DBT|CRD|PIN|SIG|CARD|NBR|TRANS(?:ACTION)?|EFF\.?|DATE|TIME|DES:|INDN:|WEB(?: ID)?:|CCD(?: ID)?:)\b", re.I), " "),
    (re.compile(r"\b(?:VISA|MASTERCARD|AMEX|DISC(?:OVER)?|DDA)\b", re.I), " "),
    (re.compile(r"\b(?:POS|PIN|MAC|MWBTGD|EXA|SIG|DDA|ACH)\b", re.I), " "),
    
    # 6) Canonicalize common merchants (keep signal)
    (re.compile(r"\bAMZN\s+MKTP\s+US\b", re.I), " amazon "),
    (re.compile(r"\bAMAZON(?:\.COM)?\b", re.I), " amazon "),
    (re.compile(r"\bAMZN\b", re.I), " amazon "),
    (re.compile(r"\bAmazon(?:\.com)?\*?[A-Z0-9-]*\b", re.I), " amazon "),
    (re.compile(r"\bAmazon\s+Prime\*?[A-Z0-9-]*\b", re.I), " amazon prime "),
    (re.compile(r"\bAPPLE(?:\.COM)?(?:\/BILL)?\b", re.I), " apple "),
    (re.compile(r"\bMicrosoft\b|\bMSBILL\.INFO\b", re.I), " microsoft "),
    (re.compile(r"\bGOOGLE\b|\bGoogle\s*Play\b|\bYouTube\b", re.I), " google "),
    (re.compile(r"\bCASH\s*APP\*?", re.I), " cash app "),
    (re.compile(r"\bPLAYSTATION(?:\s*NETWORK|NETWOR)?\b|\bSIE\b", re.I), " playstation "),
    (re.compile(r"\bEBAY\b|\beBay\b", re.I), " ebay "),
    (re.compile(r"\bAFTERPAY\b", re.I), " afterpay "),
    
    # 7) Toss order/line id styles that remain
    (re.compile(r"\bO\*\d{2}-X{5,}-\d{2,}\b", re.I), " "),
    (re.compile(r"\b[A-Z]{1,3}\d{2,4}\b(?=\s|$)"), " "),         # OK251, CA133, C718
    (re.compile(r"\b[0-9A-Z]{4,}\b(?= ?/ ?WA| ?/ ?US)", re.I), " "),
    
    # 8) Remove state abbreviations as stand-alone tokens (optional—comment if you want them)
    (re.compile(r"\b(AL|AK|AZ|AR|CA|CO|CT|DC|DE|FL|GA|HI|IA|ID|IL|IN|KS|KY|LA|MA|MD|ME|MI|MN|MO|MS|MT|NC|ND|NE|NH|NJ|NM|NV|NY|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VA|VT|WA|WI|WV|WY)\b"), " "),
    
    # 9) Punctuation/space tidy
    (re.compile(r"[|#%*()_=+;,/\\]+"), " "),
    (re.compile(r"[-]{2,}"), " "),
    (re.compile(r"\s{2,}"), " "),
    (re.compile(r"^\s+|\s+$"), ""),                               # trim
]

def clean_memo_series(s: pd.Series) -> pd.Series:
    out = s.astype(str)
    for pat, repl in RULES:
        out = out.str.replace(pat, repl, regex=True)
    # Lowercase & final collapse
    out = out.str.lower().str.replace(r"\s{2,}", " ", regex=True).str.strip()
    return out

hi = Diff.reset_index()
hi['memo_clean'] = clean_memo_series(hi['memo'])

In [121]:
hi

,index,memo,memo_clean
0,2,TST* Casa Del Rio - Exp Fairlawn OH 09/24,tst casa del rio - exp fairlawn
1,4,Buffalo Wild Wings,buffalo wild wings
2,6,Oculus CA 04/16,oculus
3,7,LOS GIRASOLES STOW OH 03/08,los girasoles stow
4,8,BUZZIS LAUNDRY 1 OH 03/28,buzzis laundry 1
...,...,...,...
1306447,2597457,DEBIT CARD WITHDRAWAL PURCHASEAmazon Prime*TI4...,purchaseamazon prime ti40l27r3 purch 15 34
1306448,2597462,POS WITHDRAWALAZ LOT QUIKTRIP XXXX XXXX E INDI...,withdrawalaz lot quiktrip e indian school rd p...
1306449,2597465,POS WITHDRAWALWAL-MART #XXXX XXXX E MCKELLIPS ...,withdrawalwal-mart e mckellips rd mesa 15 mcc
1306450,2597468,WITHDRAWAL Salt River ProjeTYPE: ONLINE PMT CO...,salt river projetype: online pmt :salt river p...


In [28]:
k = set()
for i in GM:
    words = i.split(" ")
    for j in words:
        k.add(j)

In [29]:
k

{'',
 '11/09/XXXX',
 'Amazon.com*YK11N0B',
 'XXXXXWX2YND6',
 'AMAZON.COM*1NXXXXZ70',
 '2CO.COM*slideupli',
 'Ganna',
 'LEROY',
 'SHOPS\xa0BUTTE',
 'DIGITAL*GV5T',
 'FAMI',
 'LZJ1ONJF',
 'PlaystationNetwork',
 'US*2G3BUXXXX',
 'US*RP2EF17S3',
 'Amazon.com*251OE72',
 '04:14:28',
 'AMAZON.COM*1Z73C5S',
 'LIBERTAS.ORG',
 'BILLINGTREE',
 'ATLANTICBROADBAND',
 'US*OP4GG5GI3',
 'DIGITAL*XXXXR7M60',
 'ALXXXXNPI',
 'Milford',
 'AMAZON.COM*W851S01A3',
 'TORIN',
 'DIGITAL*0T1WG8JQ3',
 'AMAZON.COM*P21C57E53',
 'MARLAND',
 'GRI',
 'C1ABXXXXXXXXXXX',
 'Amazon.com*1V9C01IR0',
 'AMAZON.COM*3O8FZ8UE3',
 'Gbeatz',
 '#283',
 '16:32:26)',
 'US*276H6',
 'EISENZIMMER',
 'AMAZON.COM*F20AC78P3',
 'US*1A7G',
 'AMAZON.COM*1O7X10MT2',
 '326',
 'Afterpay.com',
 "XXXXsam's",
 'AUTOBIDMASTER',
 'Amazon.com*XL3JJ62R3',
 'AMAZON.COM*1K9FD7IG0',
 'Z51RCXE1FQ0X',
 'XXXXAmazon.com*9K8',
 'AMAZON.COM*RA39D56L3',
 'AMAZON.COM*258KV47S0',
 'US*291SG',
 'AMAZON.COM*1V03I9IC0',
 'Y57HJ02S3',
 'AMZNFreeTime*2T3X98X82',
 '3/6'

In [2]:
Outflows['amount'] = -1 * Outflows['amount']

NameError: name 'Outflows' is not defined

In [4]:
Outflows

,prism_consumer_id,prism_account_id,memo,amount,posted_date,category
0,0,acc_0,LOAN,-900.60,2022-07-05,LOAN
1,0,acc_0,ATM_CASH,-80.00,2022-03-25,ATM_CASH
2,0,acc_0,TST* Casa Del Rio - Exp Fairlawn OH 09/24,-18.42,2022-09-26,FOOD_AND_BEVERAGES
3,0,acc_0,LOAN,-634.00,2023-01-10,LOAN
4,0,acc_0,Buffalo Wild Wings,-26.47,2022-09-12,FOOD_AND_BEVERAGES
...,...,...,...,...,...,...
2597483,5941,acc_9524,ATM_CASH,-8.42,2023-01-25,ATM_CASH
2597484,5941,acc_9524,ATM_CASH,-2.06,2023-01-25,ATM_CASH
2597485,5941,acc_9524,ATM_CASH,-262.88,2023-01-25,ATM_CASH
2597486,5941,acc_9524,ATM_CASH,-10.00,2023-01-25,ATM_CASH


In [5]:
Cash_Flows = pd.concat([Outflows, Inflows])

In [6]:
testing = Cash_Flows.set_index("prism_consumer_id")

In [7]:
train, test = train_test_split(testing, test_size=0.25)

In [8]:
test

,prism_account_id,memo,amount,posted_date,category
prism_consumer_id,,,,,
3154,acc_6737,SELF_TRANSFER,65.00,2022-10-13,SELF_TRANSFER
323,acc_960,ESSENTIAL_SERVICES,-778.50,2022-04-01,ESSENTIAL_SERVICES
5377,acc_8960,Debit: PIN purchase from XXXXXT21 WALGREENS ST...,-22.03,2021-04-21,GENERAL_MERCHANDISE
854,acc_2463,POS Debit - Visa Check Card XXXX - AMAZON.COM*...,-32.06,2021-05-07,GENERAL_MERCHANDISE
5877,acc_9460,REFUND,139.97,2022-07-13,REFUND
...,...,...,...,...,...
4178,acc_7761,PAYPAL INST XFER XXXXXX APPLE.COM BILL DEENA S...,-549.96,2022-03-29,GENERAL_MERCHANDISE
1048,acc_2965,EXTERNAL_TRANSFER,-16.67,2021-09-07,EXTERNAL_TRANSFER
2156,acc_5739,Lowe's,-6.85,2022-11-23,GENERAL_MERCHANDISE


In [9]:
ids = list(set(testing.index))

In [10]:
import pandas as pd

topN = (
    Outflows.groupby("category")["memo"]
    .value_counts()                          # counts per (category, memo)
    .groupby(level=0, group_keys=False)
    .head(15)                                # top 15 per category
    .rename("count")
    .reset_index()
)

# optional rank within each category
topN["rank"] = topN.groupby("category")["count"].rank("first", ascending=False).astype(int)
topN = topN.sort_values(["category", "rank"])

In [11]:
(Outflows[Outflows['category'] == 'UNCATEGORIZED'].shape[0]) / Outflows.shape[0]

0.04520097879181732

In [12]:
example = Outflows[Outflows['category'] == 'UNCATEGORIZED'].groupby('prism_account_id').count().sort_values(by = 'memo', ascending = False).head(1000)

In [13]:
example["prism_consumer_id"] /= (Outflows[Outflows['category'] == 'UNCATEGORIZED'].shape[0])

In [14]:
example

,prism_consumer_id,memo,amount,posted_date,category
prism_account_id,,,,,
acc_3499,0.005928,696,696,696,696
acc_7806,0.005110,600,600,600,600
acc_9332,0.005093,598,598,598,598
acc_8960,0.004403,517,517,517,517
acc_8709,0.004225,496,496,496,496
...,...,...,...,...,...
acc_7611,0.000298,35,35,35,35
acc_5325,0.000298,35,35,35,35
acc_7867,0.000298,35,35,35,35


In [15]:
in_cat = list(Inflows["category"].value_counts().index)
inside = Inflows[["category", "memo"]].value_counts()
output_inflow = {}
for i in in_cat:
    output_inflow[i] = list(inside[i].head(15).index)

In [16]:
output_inflow

{'EXTERNAL_TRANSFER': ['EXTERNAL_TRANSFER'],
 'SELF_TRANSFER': ['SELF_TRANSFER'],
 'DEPOSIT': ['DEPOSIT'],
 'MISCELLANEOUS': ['MISCELLANEOUS'],
 'PAYCHECK': ['PAYCHECK'],
 'PAYCHECK_PLACEHOLDER': ['PAYCHECK_PLACEHOLDER'],
 'REFUND': ['REFUND'],
 'INVESTMENT_INCOME': ['INVESTMENT_INCOME'],
 'SMALL_DOLLAR_ADVANCE': ['SMALL_DOLLAR_ADVANCE'],
 'OTHER_BENEFITS': ['OTHER_BENEFITS'],
 'TAX': ['TAX'],
 'LOAN': ['LOAN'],
 'UNEMPLOYMENT_BENEFITS': ['UNEMPLOYMENT_BENEFITS'],
 'INSURANCE': ['INSURANCE']}

In [17]:
testing.describe()

,amount
count,3.110603e+06
mean,6.883868e-03
std,2.672358e+03
min,-6.548532e+05
25%,-5.086000e+01
50%,-1.681000e+01
75%,-3.000000e+00
max,1.154966e+06


In [18]:
dates = Cash_Flows["posted_date"] 

In [19]:
dates.reset_index().loc[0]["posted_date"]

datetime.date(2022, 7, 5)

In [20]:
daily = Cash_Flows.set_index("prism_consumer_id").sort_values(by = "posted_date")
daily['posted_date'] = pd.to_datetime(daily['posted_date'])

In [21]:
daily.loc[1603]

,prism_account_id,memo,amount,posted_date,category
prism_consumer_id,,,,,
1603,acc_4440,ACCOUNT_FEES,-14.00,2017-12-29,ACCOUNT_FEES
1603,acc_4443,ACCOUNT_FEES,-10.00,2017-12-29,ACCOUNT_FEES
1603,acc_4444,PAYCHECK_PLACEHOLDER,6481.66,2017-12-29,PAYCHECK_PLACEHOLDER
1603,acc_4444,ACCOUNT_FEES,-2.50,2018-01-02,ACCOUNT_FEES
1603,acc_4444,ATM_CASH,-302.50,2018-01-02,ATM_CASH
...,...,...,...,...,...
1603,acc_4441,CREDIT_CARD_PAYMENT,-7029.40,2022-06-21,CREDIT_CARD_PAYMENT
1603,acc_4445,EXTERNAL_TRANSFER,-20000.00,2022-06-21,EXTERNAL_TRANSFER
1603,acc_4441,CREDIT_CARD_PAYMENT,-6343.92,2022-06-21,CREDIT_CARD_PAYMENT


In [ ]:
net_cash = {}
for i in list(daily.index):
    net_cash[i] = daily.loc[i].groupby(pd.Grouper(key='posted_date', freq='MS'))['amount'].sum().reset_index(name='amount')
         

In [ ]:
net_cash